In [1]:
import warnings
import pandas as pd
import numpy as np

In [244]:
data = pd.read_csv("cleandata.csv")

In [245]:
data

,Unnamed: 0,loan_id,user_id,total_loan,year_of_loan,interest,monthly_payment,class,employer_type,industry,...,f3,f4,early_return,early_return_amount,early_return_amount_3mon,isDefault,issue_date_month,issue_date_dayofweek,earliesCreditMon,earliesCreditYear
0,0,1040418,240418,31818.181820,3,11.466,1174.91,3,3,13,...,5.0,4.0,3,9927,0.000000,0.0,10,5,12,2001
1,1,1025197,225197,28000.000000,5,16.841,670.69,3,3,13,...,45.0,22.0,0,0,0.000000,0.0,6,5,4,1990
2,2,1009360,209360,17272.727270,3,8.900,603.32,1,3,3,...,28.0,19.0,0,0,0.000000,0.0,1,2,10,1991
3,3,1039708,239708,20000.000000,3,4.788,602.30,1,1,10,...,15.0,9.0,0,0,0.000000,0.0,7,2,6,2001
4,4,1027483,227483,15272.727270,3,12.790,470.31,3,3,2,...,15.0,4.0,0,0,0.000000,0.0,7,4,5,2001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
764851,764851,1008856,208856,9454.545455,5,12.015,183.47,3,3,2,...,14.0,8.0,0,0,0.000000,NaN,1,5,7,2001
764852,764852,1016651,216651,5500.000000,3,7.970,172.28,1,3,8,...,5.0,3.0,3,1564,0.000000,NaN,5,0,4,2001
764853,764853,1024140,224140,30545.454550,3,8.900,889.09,1,0,8,...,20.0,14.0,2,5456,1510.892308,NaN,12,6,10,1986
764854,764854,1014316,214316,4090.909091,3,6.030,152.18,1,3,10,...,10.0,10.0,3,223,41.169231,NaN,9,5,3,1999


In [247]:
train = data[data["isDefault"].notna()]
test = data[data["isDefault"].isna()]

In [248]:
features = [f for f in train.columns if f not in ['loan_id', 'user_id', 'isDefault']]

In [249]:
ordata_preds = np.zeros(train.shape[0])
preds = np.zeros(test.shape[0])

In [250]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold

In [251]:
import gc
from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, average_precision_score
from dateutil.relativedelta import relativedelta

In [252]:
y = train["isDefault"]

In [194]:
kfold = StratifiedKFold(n_splits = 10,shuffle=True ,random_state=0)

In [253]:
for k, (train_index, valuation_index) in enumerate(kfold.split(train,y)):
    train_x, train_y = train[features].iloc[train_index], y.iloc[train_index]
    valuation_x, valuation_y = train[features].iloc[valuation_index], y.iloc[valuation_index]
    lr = LogisticRegression(C=1.0, random_state=0) #Logistic model
    lr.fit(train_x,train_y)
    ordata_preds[valuation_index]  = lr.predict_proba(valuation_x)[:,1]
    print("Fold %2d AUC : %.6f" % (k + 1, roc_auc_score(valuation_y, ordata_preds[valuation_index])))
    preds  += lr.predict_proba(test[features])[:,1] / kfold.n_splits
    del lr, train_x, train_y, valuation_x, valuation_y 
    gc.collect()
print(" Full AUC score %.6f" % roc_auc_score(y, ordata_preds)) 


C:\Users\LDW\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold  1 AUC : 0.653953


C:\Users\LDW\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold  2 AUC : 0.658334


C:\Users\LDW\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


...

Fold  8 AUC : 0.657155


C:\Users\LDW\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold  9 AUC : 0.648387


C:\Users\LDW\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Fold 10 AUC : 0.654529
 Full AUC score 0.650878


In [254]:
test["isDefault"] = preds

<ipython-input-254-4cacd7473762>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test["isDefault"] = preds


In [255]:
results = test[["loan_id","isDefault"]]

In [256]:
results = results.rename(columns={"loan_id":"id"})

In [257]:
results

,id,isDefault
759856,1000575,1.859374e-04
759857,1028125,1.005656e-07
759858,1010694,6.607136e-19
759859,1026712,1.647989e-15
759860,1002895,2.008745e-12
...,...,...
764851,1008856,2.595504e-01
764852,1016651,1.005893e-06
764853,1024140,1.147550e-18
764854,1014316,1.603030e-02


In [258]:
results.to_csv("logisticmodelfinal.csv",index=False)